<a href="https://colab.research.google.com/github/ezzeddinegasmi/BreakOut_sb3_A2C/blob/main/A2C_FAUX_faux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -r requirements.txt
! pip install torch
! pip install gymnasium
! pip install box2d-py
! pip install stable-baselines3[extra]
! pip install wandb
! pip install numpy
! pip install matplotlib
! pip install time
! pip install seaborn
! pip install tqdm
! pip install pygame
! pip install opencv-python

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (box2d-py)
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
pip show pygame


Name: pygame
Version: 2.6.1
Summary: Python Game Development
Home-page: https://www.pygame.org
Author: A community project.
Author-email: pygame@pygame.org
License: LGPL
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: dopamine_rl


In [ ]:
# @title Titre par défaut
pip show -r requirements.txt


SyntaxError: invalid syntax (<ipython-input-11-0f4742bd04fa>, line 2)

## Configuration for Colab

In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install gymnasium==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.1.1
    Uninstalling gymnasium-1.1.1:
      Successfully uninstalled gymnasium-1.1.1


## import module

In [ ]:
pip install stable-baselines3[extra] gymnasium wandb numpy

In [ ]:
import gymnasium as gym
from gymnasium.envs.registration import register

try:
    register(
        id='BreakoutNoFrameskip-v4',
        entry_point='gymnasium.envs.atari:AtariEnv',
        kwargs={'game': 'breakout', 'obs_type': 'rgb', 'frameskip': 1},
        max_episode_steps=10000,
        nondeterministic=False,
    )
    print("Environment registered successfully.")
except Exception as e:
    print(f"Error registering environment: {e}")

Environment registered successfully.


In [ ]:
#! pip install atari-py
#! pip install gym[atari,accept-rom-license]
#! pip install --upgrade gym
#env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=config.n_envs, seed=config.seed)
from stable_baselines3.common.env_util import make_atari_env
!pip install ale-py==0.8.1 # Install ale-py
!pip install gym[accept-rom-license,atari] # Install gym[atari] and accept the ROM license
import gymnasium as gym # Import gymnasium
import config # Import the config module

# Instead of gym.make, use make_atari_env from stable_baselines3
#env = make_atari_env('BreakoutNoFrameskip-v4', n_envs=1, seed=0) # Assuming n_envs and seed are defined
env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=config.n_envs, seed=config.seed)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.11.0
    Uninstalling ale-py-0.11.0:
      Successfully uninstalled ale-py-0.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires ale-py>=0.10.1, but you have ale-py 0.8.1 which is incompatible.
INFO: pip is looking at multiple versions of gym[accept-rom-license,atari] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 31.3 MB/s eta 0:00:00
  Installing build dependencies ... do

ModuleNotFoundError: No module named 'config'

In [ ]:
### Train A2C

from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.atari_wrappers import AtariWrapper
import gymnasium as gym
import torch
import config
import wandb
from wandb.integration.sb3 import WandbCallback
from utils import make_env, unzip_file, CustomWandbCallback, RewardLogger
import os
from stable_baselines3.common.utils import get_latest_run_id
import tensorboard as tb
from tqdm import tqdm
'''
Set up the appropriate directories for logging and saving the model
'''
os.makedirs(config.log_dir, exist_ok=True)
os.makedirs(config.save_path, exist_ok=True)

#Create the callback that logs the mean reward of the last 100 episodes to wandb
custom_callback = CustomWandbCallback(config.check_freq, config.save_path)


'''
Set up loging to wandb
'''
#Set wandb to log the training process
if config.log_to_wandb:
    wandb.init(project=config.project_train, entity = config.entity, name=config.name_train, notes=config.notes, sync_tensorboard=config.sync_tensorboard)
    #wandb_callback is a callback that logs the training process to wandb, this is done because wandb.watch() does not work with sb3
    wandb_callback = WandbCallback()


'''
Set up the environment
'''
# Create multiple environments and wrap them correctly
env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=config.n_envs, seed=config.seed)
env = VecFrameStack(env, n_stack=config.n_stack)

'''
Set up the model
'''
#Create the model with the parameters specified in config.py, go to config.py to see the meaning of each parameter in detail
model = A2C(
            policy = config.policy
            , env = env
            , verbose=config.verbose
            , tensorboard_log=config.log_dir
            , seed=config.seed
            , policy_kwargs=config.policy_kwargs
            , sde_sample_freq=config.sde_sample_freq
            , normalize_advantage=config.normalize_advantage
            , stats_window_size=config.stats_window_size
            , _init_setup_model=config._init_setup_model
            , device=config.device
            , learning_rate=config.learning_rate
            , n_steps=config.n_steps
            , gamma=config.gamma
            , gae_lambda=config.gae_lambda
            , ent_coef=config.ent_coef
            , vf_coef=config.vf_coef
            , max_grad_norm=config.max_grad_norm
            , rms_prop_eps=config.rms_prop_eps
            , use_rms_prop=config.use_rms_prop
            , use_sde=config.use_sde,
            )

#Load the model if config.pretrained is set to True in config.py
if config.pretrained:
    model = A2C.load(config.saved_model_path, env=env, verbose=config.verbose, tensorboard_log=config.log_dir)
    #Unzip the file a2c_Breakout_1M.zip and store the unzipped files in the folder a2c_Breakout_unzipped
    unzip_file(config.saved_model_path, config.unzip_file_path)
    model.policy.load_state_dict(torch.load(os.path.join(config.unzip_file_path, "policy.pth")))
    model.policy.optimizer.load_state_dict(torch.load(os.path.join(config.unzip_file_path, "policy.optimizer.pth")))



'''
Train the model and save it
'''
#model.learn will train the model for 1e6 timesteps, timestep is the number of actions taken by the agent,
# in a game like breakout, the agent takes an action every frame, then the number of timesteps is the number of frames,
# which is the number of frames in 1 game multiplied by the number of games played.
#The average number of frames in 1 game is 1000, so 1e6 timesteps is 1000 games more or less.
#log_interval is the number of timesteps between each log, in this case, the training process will be logged every 100 timesteps.
#callback is a callback that logs the training process to wandb, this is done because wandb.watch() does not work with sb3

if config.log_to_wandb:
    model.learn(total_timesteps=config.total_timesteps, log_interval=config.log_interval, callback=[wandb_callback, custom_callback], progress_bar=True)
else:
    model.learn(total_timesteps=config.total_timesteps, log_interval=config.log_interval, callback=custom_callback, progress_bar=True)
#Save the model
model.save(config.saved_model_path[:-4]) #remove the .zip extension from the path


'''
Close the environment and finish the logging
'''
env.close()
if config.log_to_wandb:
    wandb.finish()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import random
from collections import deque
from typing import Deque, Dict, List, Tupl
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.distributions import Normal

In [ ]:
def initialize_uniformly(layer: nn.Linear, init_w: float = 3e-3):
    """Initialize the weights and bias in [-init_w, init_w]."""
    layer.weight.data.uniform_(-init_w, init_w)
    layer.bias.data.uniform_(-init_w, init_w)


class Actor(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        """Initialize."""
        super(Actor, self).__init__()

        self.hidden1 = nn.Linear(in_dim, 128)
        self.mu_layer = nn.Linear(128, out_dim)
        self.log_std_layer = nn.Linear(128, out_dim)

        initialize_uniformly(self.mu_layer)
        initialize_uniformly(self.log_std_layer)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        x = F.relu(self.hidden1(state))

        mu = torch.tanh(self.mu_layer(x)) * 2
        log_std = F.softplus(self.log_std_layer(x))
        std = torch.exp(log_std)

        dist = Normal(mu, std)
        action = dist.sample()

        return action, dist


class Critic(nn.Module):
    def __init__(self, in_dim: int):
        """Initialize."""
        super(Critic, self).__init__()

        self.hidden1 = nn.Linear(in_dim, 128)
        self.out = nn.Linear(128, 1)

        initialize_uniformly(self.out)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        x = F.relu(self.hidden1(state))
        value = self.out(x)

        return value

NameError: name 'nn' is not defined

## GAE

*GAE* help to reduce variance while maintaining a proper level of bias. By adjusting parameters $\lambda \in [0, 1]$ and $\gamma \in [0, 1]$. Please see [the paper](https://arxiv.org/pdf/1506.02438) for detailed description.

In [ ]:
def compute_gae(
    next_value: list, rewards: list, masks: list, values: list, gamma: float, tau: float
) -> List:
    """Compute gae."""
    values = values + [next_value]
    gae = 0
    returns: Deque[float] = deque()

    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.appendleft(gae + values[step])

    return list(returns)

NameError: name 'List' is not defined

## Network
We will use two separated networks for actor and critic respectively. The actor network consists of one fully connected hidden layer with ReLU branched out two fully connected output layers for mean and standard deviation of Normal distribution. Pendulum-v0 has only one action which has a range from -2 to 2. In order to fit the range, the actor outputs the mean value that is multiplied by 2 after tanh. On the one hand, the critic network has two fully connected layers as a hidden layer (ReLU) and an output layer. One thing to note is that we initialize the last layers' weights and biases as uniformly distributed.

In [ ]:
#BreakOut_sb3_A2C

In [ ]:
#BreakOut_sb3_A2C/config.py

In [ ]:
import os
from stable_baselines3.common.utils import get_latest_run_id
import torch


'''FILE TO STORE ALL THE CONFIGURATION VARIABLES'''

#pretrained is a boolean that indicates if a pretrained model will be loaded
pretrained = False # Set to True if you want to load a pretrained model

#check_freq is the frequency at which the callback is called, in this case, the callback is called every 2000 timesteps
check_freq = 2000

#save_path is the path where the best model will be saved
save_path = './a2c_Breakout_1M_save_path'

#log_dir is the path where the logs will be saved
log_dir = './log_dir'


'''
Hyperparameters of the model {learning_rate, gamma, device, n_steps, gae_lambda, ent_coef, vf_coef, max_grad_norm, rms_prop_eps, use_rms_prop, use_sde, sde_sample_freq, normalize_advantage}
'''
#policy is the policy of the model, in this case, the model will use a convolutional neural network
policy = "CnnPolicy"

#learning_rate is the learning rate of the model
learning_rate =5e-4   #first trial: 5e-4   #second trial: 1e-4  #third trial: 1e-3  #fourth trial: 5e-5 #fifth trial: 5e-5 gamma = 0.90 #sixth trial: 1e-4 gamma = 0.90 #seventh trial: 5e-4 gamma = 0.90


#gamma is the discount factor
gamma = 0.99

#device is the device where the model will be trained, if cuda is available, the model will be trained in the gpu, otherwise, it will be trained in the cpu
device = "cuda" if torch.cuda.is_available() else "cpu"

#n_steps is the number of steps taken by the model before updating the parameters
n_steps = 24

#gae_lambda is the lambda parameter of the generalized advantage estimation, set to 1 to disable it
gae_lambda = 1

#ent_coef is the entropy coefficient, set to 0 to disable it
ent_coef = 0

#vf_coef is the value function coefficient, If we set it to 0.5, then the value function loss will be half the policy loss
vf_coef = 0.5

#max_grad_norm is the maximum value for the gradient clipping
max_grad_norm = 0.5

#rms_prop_eps is the RMSProp optimizer epsilon parameter, which is a small value added to the denominator to avoid division by zero
rms_prop_eps = 0.00001

#use_rms_prop is a boolean that indicates if the RMSProp optimizer will be used
use_rms_prop = True

#use_sde is a boolean that indicates if the stochastic differential equation will be used
#The stochastic differential equation is a method to add noise to the actions taken by the agent to improve exploration
use_sde = False

#sde_sample_freq is the frequency at which the noise is added to the actions. If set to -1, the noise will be added every timestep
sde_sample_freq = -1

#normalize_advantage is a boolean that indicates if the advantage will be normalized, by normalizing the advantage,
# the variance of the advantage is reduced, this is done to improve the training process because the advantage is used to calculate the policy loss
normalize_advantage = False

#stats_window_size is the size of the window used to calculate the mean and standard deviation of the advantage
stats_window_size = 100

#tensorboard_log is the path where the tensorboard logs will be saved, in our case, the logs will be saved in the log_dir
tensorboard_log = log_dir

#policy_kwargs is a dictionary with the keyword arguments for the policy. If None, it will use the default arguments
policy_kwargs = None

#verbose is the verbosity level: 0 no output, 1 info, 2 debug
verbose = 2

#seed is the seed for the pseudo random number generator used by the model. It is set to None to use a random seed,
# and set to 0 to use a fixed seed for reproducibility
seed = None

#_init_setup_model is a boolean that indicates if the model will be initialized after being created, set to True to initialize the model
_init_setup_model = True

#total_timesteps is the total number of timesteps that the model will be trained. In this case, the model will be trained for 1e7 timesteps
#Take into account that the number of timesteps is not the number of episodes, in a game like breakout, the agent takes an action every frame,
# then the number of timesteps is the number of frames, which is the number of frames in 1 game multiplied by the number of games played.
#The average number of frames in 1 game is 1000, so 1e7 timesteps is 1000 games more or less.
total_timesteps = int(3e7)

#log_interval is the number of timesteps between each log, in this case, the training process will be logged every 100 timesteps.
log_interval = 100

'''
Saved model path
'''
saved_model_path = "./a2c_Breakout_30M_lr_5e-4_gamma_90.zip"
unzip_file_path = "./a2c_Breakout_30M_lr_5e-4_gamma_90_unzipped"
'''
Environment variables
'''
#n_stack is the number of frames stacked together to form the input to the model
n_stack = 4
#n_envs is the number of environments that will be run in parallel
n_envs = 4


'''
Wandb configuration
'''

#log_to_wandb is a boolean that indicates if the training process will be logged to wandb
log_to_wandb = False

#project is the name of the project in wandb
project_train = "BREAKOUT_SB3_BENCHMARK"
project_test = "breakout-a2c-test"

#entity is the name of the team in wandb
entity = "ai42"

#name is the name of the run in wandb
name_train = "a2c_breakout_lr_5e-4_gamma_90"
name_test = "a2c_breakout_test"
#notes is a description of the run
notes = "a2c_breakout with parameters: {}".format(locals()) #locals() returns a dictionary with all the local variables, in this case, all the variables in this file
#sync_tensorboard is a boolean that indicates if the tensorboard logs will be synced to wandb
sync_tensorboard = True


'''
Test configuration
'''
test_episodes = 100

In [ ]:
#BreakOut_sb3_A2C/train_A2C.py

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.atari_wrappers import AtariWrapper
import gymnasium as gym
import torch
import config
import wandb
from wandb.integration.sb3 import WandbCallback
from utils import make_env, unzip_file, CustomWandbCallback, RewardLogger
import os
from stable_baselines3.common.utils import get_latest_run_id
import tensorboard as tb
from tqdm import tqdm


'''
Set up the appropriate directories for logging and saving the model
'''
os.makedirs(config.log_dir, exist_ok=True)
os.makedirs(config.save_path, exist_ok=True)

#Create the callback that logs the mean reward of the last 100 episodes to wandb
custom_callback = CustomWandbCallback(config.check_freq, config.save_path)


'''
Set up loging to wandb
'''
#Set wandb to log the training process
if config.log_to_wandb:
    wandb.init(project=config.project_train, entity = config.entity, name=config.name_train, notes=config.notes, sync_tensorboard=config.sync_tensorboard)
    #wandb_callback is a callback that logs the training process to wandb, this is done because wandb.watch() does not work with sb3
    wandb_callback = WandbCallback()


'''
Set up the environment
'''
# Create multiple environments and wrap them correctly
env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=config.n_envs, seed=config.seed)
env = VecFrameStack(env, n_stack=config.n_stack)


'''
Set up the model
'''
#Create the model with the parameters specified in config.py, go to config.py to see the meaning of each parameter in detail
model = A2C(
            policy = config.policy
            , env = env
            , verbose=config.verbose
            , tensorboard_log=config.log_dir
            , seed=config.seed
            , policy_kwargs=config.policy_kwargs
            , sde_sample_freq=config.sde_sample_freq
            , normalize_advantage=config.normalize_advantage
            , stats_window_size=config.stats_window_size
            , _init_setup_model=config._init_setup_model
            , device=config.device
            , learning_rate=config.learning_rate
            , n_steps=config.n_steps
            , gamma=config.gamma
            , gae_lambda=config.gae_lambda
            , ent_coef=config.ent_coef
            , vf_coef=config.vf_coef
            , max_grad_norm=config.max_grad_norm
            , rms_prop_eps=config.rms_prop_eps
            , use_rms_prop=config.use_rms_prop
            , use_sde=config.use_sde,
            )

#Load the model if config.pretrained is set to True in config.py
if config.pretrained:
    model = A2C.load(config.saved_model_path, env=env, verbose=config.verbose, tensorboard_log=config.log_dir)
    #Unzip the file a2c_Breakout_1M.zip and store the unzipped files in the folder a2c_Breakout_unzipped
    unzip_file(config.saved_model_path, config.unzip_file_path)
    model.policy.load_state_dict(torch.load(os.path.join(config.unzip_file_path, "policy.pth")))
    model.policy.optimizer.load_state_dict(torch.load(os.path.join(config.unzip_file_path, "policy.optimizer.pth")))



'''
Train the model and save it
'''
#model.learn will train the model for 1e6 timesteps, timestep is the number of actions taken by the agent,
# in a game like breakout, the agent takes an action every frame, then the number of timesteps is the number of frames,
# which is the number of frames in 1 game multiplied by the number of games played.
#The average number of frames in 1 game is 1000, so 1e6 timesteps is 1000 games more or less.
#log_interval is the number of timesteps between each log, in this case, the training process will be logged every 100 timesteps.
#callback is a callback that logs the training process to wandb, this is done because wandb.watch() does not work with sb3

if config.log_to_wandb:
    model.learn(total_timesteps=config.total_timesteps, log_interval=config.log_interval, callback=[wandb_callback, custom_callback], progress_bar=True)
else:
    model.learn(total_timesteps=config.total_timesteps, log_interval=config.log_interval, callback=custom_callback, progress_bar=True)
#Save the model
model.save(config.saved_model_path[:-4]) #remove the .zip extension from the path


'''
Close the environment and finish the logging
'''
env.close()
if config.log_to_wandb:
    wandb.finish()

ModuleNotFoundError: No module named 'config'

## A2C Agent
Here is a summary of A2CAgent class.

| Method           | Note                                                 |
|---               |---                                                   |
|select_action     | select an action from the input state.               |
|step              | take an action and return the response of the env.   |
|update_model      | update the model by gradient descent.                |
|train             | train the agent during num_frames.                   |
|test              | test the agent (1 episode).                          |
|_plot             | plot the training progresses.                        |


In [ ]:
class A2CAgent:
    """A2CAgent interacting with environment.

    Atribute:
        env (gym.Env): openAI Gym environment
        gamma (float): discount factor
        entropy_weight (float): rate of weighting entropy into the loss function
        device (torch.device): cpu / gpu
        actor (nn.Module): target actor model to select actions
        critic (nn.Module): critic model to predict state values
        actor_optimizer (optim.Optimizer) : optimizer of actor
        critic_optimizer (optim.Optimizer) : optimizer of critic
        transition (list): temporory storage for the recent transition
        total_step (int): total step numbers
        is_test (bool): flag to show the current mode (train / test)
        seed (int): random seed
    """

    def __init__(self, env: gym.Env, gamma: float, entropy_weight: float, seed: int = 777):
        """Initialize."""
        self.env = env
        self.gamma = gamma
        self.entropy_weight = entropy_weight
        self.seed = seed

        # device: cpu / gpu
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)

        # networks
        obs_dim = env.observation_space.shape[0]
        action_dim = env.action_space.shape[0]
        self.actor = Actor(obs_dim, action_dim).to(self.device)
        self.critic = Critic(obs_dim).to(self.device)

        # optimizer
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-4)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=1e-3)

        # transition (state, log_prob, next_state, reward, done)
        self.transition: list = list()

        # total steps count
        self.total_step = 0

        # mode: train / test
        self.is_test = False

    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        state = torch.FloatTensor(state).to(self.device)
        action, dist = self.actor(state)
        selected_action = dist.mean if self.is_test else action

        if not self.is_test:
            log_prob = dist.log_prob(selected_action).sum(dim=-1)
            self.transition = [state, log_prob]

        return selected_action.clamp(-2.0, 2.0).cpu().detach().numpy()

    def step(self, action: np.ndarray) -> Tuple[np.ndarray, np.float64, bool]:
        """Take an action and return the response of the env."""
        next_state, reward, terminated, truncated, _ = self.env.step(action)
        done = terminated or truncated

        if not self.is_test:
            self.transition.extend([next_state, reward, done])

        return next_state, reward, done

    def update_model(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """Update the model by gradient descent."""
        state, log_prob, next_state, reward, done = self.transition

        # Q_t   = r + gamma * V(s_{t+1})  if state != Terminal
        #       = r                       otherwise
        mask = 1 - done
        next_state = torch.FloatTensor(next_state).to(self.device)
        pred_value = self.critic(state)
        targ_value = reward + self.gamma * self.critic(next_state) * mask
        value_loss = F.smooth_l1_loss(pred_value, targ_value.detach())

        # update value
        self.critic_optimizer.zero_grad()
        value_loss.backward()
        self.critic_optimizer.step()

        # advantage = Q_t - V(s_t)
        advantage = (targ_value - pred_value).detach()  # not backpropagated
        policy_loss = -advantage * log_prob
        policy_loss += self.entropy_weight * -log_prob  # entropy maximization

        # update policy
        self.actor_optimizer.zero_grad()
        policy_loss.backward()
        self.actor_optimizer.step()

        return policy_loss.item(), value_loss.item()

    def train(self, num_frames: int, plotting_interval: int = 200):
        """Train the agent."""
        self.is_test = False

        actor_losses, critic_losses, scores = [], [], []
        state, _ = self.env.reset(seed=self.seed)
        score = 0

        for self.total_step in range(1, num_frames + 1):
            action = self.select_action(state)
            next_state, reward, done = self.step(action)

            actor_loss, critic_loss = self.update_model()
            actor_losses.append(actor_loss)
            critic_losses.append(critic_loss)

            state = next_state
            score += reward

            # if episode ends
            if done:
                state, _ = self.env.reset(seed=self.seed)
                scores.append(score)
                score = 0

            # plot
            if self.total_step % plotting_interval == 0:
                self._plot(self.total_step, scores, actor_losses, critic_losses)
        self.env.close()

    def test(self, video_folder: str):
        """Test the agent."""
        self.is_test = True

        tmp_env = self.env
        self.env = gym.wrappers.RecordVideo(self.env, video_folder=video_folder)

        state, _ = self.env.reset(seed=self.seed)
        done = False
        score = 0

        while not done:
            action = self.select_action(state)
            next_state, reward, done = self.step(action)

            state = next_state
            score += reward

        print("score: ", score)
        self.env.close()

        self.env = tmp_env

    def _plot(
        self,
        frame_idx: int,
        scores: List[float],
        actor_losses: List[float],
        critic_losses: List[float],
    ):
        """Plot the training progresses."""

        def subplot(loc: int, title: str, values: List[float]):
            plt.subplot(loc)
            plt.title(title)
            plt.plot(values)

        subplot_params = [
            (131, f"frame {frame_idx}. score: {np.mean(scores[-10:])}", scores),
            (132, "actor_loss", actor_losses),
            (133, "critic_loss", critic_losses),
        ]

        clear_output(True)
        plt.figure(figsize=(30, 5))
        for loc, title, values in subplot_params:
            subplot(loc, title, values)
        plt.show()

NameError: name 'np' is not defined

In [ ]:
def A2C_iter(
    epoch: int,
    mini_batch_size: int,
    states: torch.Tensor,
    actions: torch.Tensor,
    values: torch.Tensor,
    log_probs: torch.Tensor,
    returns: torch.Tensor,
    advantages: torch.Tensor,
):
    """Yield mini-batches."""
    batch_size = states.size(0)
    for _ in range(epoch):
        for _ in range(batch_size // mini_batch_size):
            rand_ids = np.random.choice(batch_size, mini_batch_size)
            yield states[rand_ids, :], actions[rand_ids], values[rand_ids], log_probs[
                rand_ids
            ], returns[rand_ids], advantages[rand_ids]

In [ ]:
# environment
env = gym.make("Pendulum-v1", render_mode="rgb_array")

## Set random seed

In [ ]:
def seed_torch(seed):
    torch.manual_seed(seed)
    if torch.backends.cudnn.enabled:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True


seed = 777
random.seed(seed)
np.random.seed(seed)
seed_torch(seed)

NameError: name 'random' is not defined

## Initialize

In [ ]:
num_frames = 100000
gamma = 0.90
entropy_weight = 1e-2

agent = A2CAgent(env, gamma, entropy_weight, seed)

NameError: name 'A2CAgent' is not defined

## Train

In [ ]:
agent.train(num_frames)

NameError: name 'agent' is not defined

## Test

In [ ]:
# test
video_folder = "videos/a2c"
agent.test(video_folder=video_folder)

## Render

In [ ]:
import base64
import glob
import io
import os

from IPython.display import HTML, display


def ipython_show_video(path: str) -> None:
    """Show a video at `path` within IPython Notebook."""
    if not os.path.isfile(path):
        raise NameError("Cannot access: {}".format(path))

    video = io.open(path, "r+b").read()
    encoded = base64.b64encode(video)

    display(
        HTML(
            data="""
        <video width="320" height="240" alt="test" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
        </video>
        """.format(
                encoded.decode("ascii")
            )
        )
    )


def show_latest_video(video_folder: str) -> str:
    """Show the most recently recorded video from video folder."""
    list_of_f tt le rr4iles = glob.glob(os.path.join(video_folder, "*.mp4"))
    latest_file = max(list_of_files, key=os.path.getctime)
    ipython_show_video(latest_file)
    return latest_file


latest_file = show_latest_video(video_folder=video_folder)
print("Played:", latest_file)

In [ ]:
def render(self, mode='human'):
    if not hasattr(self, 'screen'):
        import pygame
        pygame.init()
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption("Breakout Continuous")
        self.clock = pygame.time.Clock()

    self.screen.fill((0, 0, 0))
    pygame.draw.circle(self.screen, (255, 255, 255), (int(self.ball_x), int(self.ball_y)), self.ball_size)
    pygame.draw.rect(
        self.screen,
        (0, 255, 0),
        pygame.Rect(int(self.paddle_x), self.screen_height - self.paddle_height, self.paddle_width, self.paddle_height)
    )

    pygame.display.flip()
    self.clock.tick(60)

def close(self):
    if hasattr(self, 'screen'):
        import pygame
        pygame.quit()